# References

- Trustworthy Machine Learning Lecture at the University of Tübingen. 
Winter Semester 2023/2024. https://www.youtube.com/playlist?list=PL05umP7R6ij0FDHxle4CQLkzOfN-PAt7b ***
- Mucsányi B, Kirchhof M, Nguyen E, Rubinstein A, Oh SJ. Trustworthy Machine Learning 2023. ***
- Guo C, Pleiss G, Sun Y, Weinberger KQ. On Calibration of Modern Neural Networks 2017. 
- Gawlikowski J, Tassi CRN, Ali M, Lee J, Humt M, Feng J, et al. A survey of uncertainty in deep neural networks. Artif Intell Rev 2023;56:1513–89. ***
- Hüllermeier E, Waegeman W. Aleatoric and epistemic uncertainty in machine learning: an introduction to concepts and methods. Mach Learn 2021;110:457–506. ***
- Huang L, Ruan S, Xing Y, Feng M. A review of uncertainty quantification in medical image analysis: Probabilistic and non-probabilistic methods. Medical Image Analysis 2024;97:103223. ***
- Angelopoulos AN, Kohli AP, Bates S, Jordan MI, Malik J, Alshaabi T, et al. Image-to-Image Regression with Distribution-Free Uncertainty Quantification and Applications in Imaging 2022.




# Trustworthy AI 2: Uncertainty quantification

Definition: Uncertainty Quantification (UQ) is the process of quantifying the level of confidence in a model’s predictions. It helps identify how sure or unsure the model is about its outputs.

**Key Learning Objectives:**

- Understand the role of UQ in building trustworthy AI.
- Differentiate between types of uncertainty and their implications.
- Explore techniques for measuring and handling uncertainty.

# Introduction

## Why Uncertainty quantification

![](images_uq/intro.png)

- Human-in-the-Loop Decision Making

    - Goal: Efficiently allocate human resources by only requiring human intervention when the model’s confidence is low.
    - Mechanism: The model indicates uncertainty by saying, “I am not sure about the result.” In these cases, human experts can step in, assess the situation, and adjust the prediction if necessary.

- Risk Management and Safety

    - Goal: Minimize risks by ensuring the model only acts when confident.
    - Mechanism: If the model is unsure, the system can halt processing or switch to a safe fallback state to avoid potential errors.
    - Example: In a learning-based manufacturing robot for car assembly, if the robot is uncertain about its next action, it may pause or escalate to a safer state. Acting with uncertainty could lead to costly mistakes, including damaging the product.

## Example use cases

#### Example: Image search for disease

![](images_uq/intro2.png)

ref: https://blog.google/technology/health/ai-dermatology-preview-io-2021/

An image search tool for skin disease detection that flags cases with high uncertainty for physician consultation.

#### Example: Active learning

![](images_uq/intro3.png)

ref: Active Learning Literature Survey, Settles 2010

In active learning, the model identifies instances it is uncertain about and prioritizes them for labeling, reducing the need for extensive labeled datasets.

ref: https://www.gatsby.ucl.ac.uk/~balaji/balaji-uncertainty-talk-cifar-dlrl.pdf

#### Object detection pipeline

![](images_uq/intro4.png)

- Object detection pipelines, such as in Faster R-CNN, use “objectness” scores to quantify confidence for each detected object.
- The model proposes bounding boxes and provides a confidence score for each. This score helps prune unlikely boxes, improving both accuracy and efficiency.

## Example of Uncertainty



![](images_uq/uq_ex1.png)

- f(x) is output
- c(x) is probability that f(x) is correct (0 <= c(x) <= 1)

![](images_uq/uq_ex2.png)

face recognition

- c(x) can be vector of confident

![](images_uq/uq_ex3.png)

A. N. Angelopoulos et al., “Image-to-image regression with distribution-free uncertainty quantification and applications in imaging,” PMLR, https://proceedings.mlr.press/v162/angelopoulos22a.html . 

- uncertainty also can quantify as a set of value (set of possible value of image in this case)

![](images_uq/uq_ex4.png)

- uncertainty also can quantify as a set of value



### Binary Classification Task

- Given an input 𝑥, the model produces an output 𝑓(𝑥), which represents the predicted probability that 𝑥 belongs to the positive class. 
- Typically, a cut-off point of 0.5 is used.
- If 𝑓(𝑥)>0.5, classify as Positive (Class 1).

In this case, we may define confidence $c(x)$ as $max(f(x), 1-f(x))$ and uncertainty as $1- c(x)$.

![](images_uq/cx.png)



# Uncertainty evaluation

## Calibration

Calibration measures how well the predicted probabilities match the actual observed outcomes, indicating the accuracy of predictive uncertainty.

- A well-calibrated model produces probabilities that reflect true likelihoods. For example, if a model predicts a probability of 0.7 for a positive outcome, that outcome should occur approximately 70% of the time.
- Calibration helps assess the reliability of predictions, particularly useful in high-stakes applications where understanding prediction confidence is crucial.



### Evaluating calibration

### Qualitative

![](images_uq/cal.png)

Guo C, Pleiss G, Sun Y, Weinberger KQ. On Calibration of Modern Neural Networks 2017.


- Left is well-calibrated model
- Right is less well-calibrated model compare to the left



We defined the perfect calibration as:

$$P(\hat{Y}=Y | \hat{P}=p) = p$$

- $\hat{Y}$ is prediction class
- $Y$ is real class
- $\hat{P}$ is prediction probability
- $p$ is the prediction accuracy

$$P(\hat{Y}=1 | \hat{P}=0.8) = 0.8$$

This means that when the model predicts an instance as class 1 with an 80% confidence, it should be correct approximately 80% of the time. Calibration ensures that the model’s confidence aligns with real-world outcomes.

### Expected calibration error (ECE)

- ECE quantifies how far a model’s predicted probabilities are from perfect calibration.
- ECE groups predictions into intervals (e.g., 0.1 to 0.2, 0.2 to 0.3, etc.), calculates the accuracy in each interval, and computes the average difference between accuracy and predicted probability.

You have a set of all prediction $(\hat{y}_j, \hat{p}_j)$. We can calculate ECE as:

1. Group the predictions into M bins (typically M = 10) based on the confidences estimates. Define bin $B_m$ to be the set of all predictions $(\hat{y}_i, \hat{p}_i)$ for which it holds that

![](images_uq/cal_ece.png)

2. Compute the accuracy and confidence of each bin $B_m$

$$\text{acc}(B_m) = \frac{1}{|B_m|} \sum_{i \in B_m} \mathcal{1}(\hat{y_i}=y_i)$$
$$\text{conf}(B_m) = \frac{1}{|B_m|} \sum_{i \in B_m} \hat{p}_i$$

3. Compute the ECE by taking the mean over the bins weighted by the number of samples in them.

$$\text{ECE} = \frac{|B_m|}{n} \sum_{m=1}^M |\text{acc}(B_m) - \text{conf}(B_m)|$$

for $n$ is the number of all sample in the set

The ECE measures the deviation of the model’s confidence predictions from the corresponding actual accuracies on a test set. It is a weighted average of bin-wise miscalibration.

![](images_uq/ece.png)

#### On Calibration of Modern Neural Networks

Modern neural networks frequently demonstrate poor calibration compared to their predecessors. Large models with minimal regularization often struggle to generalize effectively. While many studies show that deep learning can achieve good generalization by measuring discrimination performance. This suggests that high-capacity models are not immune to overfitting; instead, overfitting tends to manifest as probabilistic error rather than as straightforward classification error.

![](images_uq/dl_cal.png)

Guo C, Pleiss G, Sun Y, Weinberger KQ. On Calibration of Modern Neural Networks 2017.


## Scoring functions

### Brier score

$$BS = \frac{1}{N}\sum_{n=1}^N (\hat{p} - y)$$

- $\hat{p}$ is predicted probability
- $y$ is the true class
- Lower Brier Score indicates better calibration and accuracy of the uncertainty estimates.

> Brier score is a measure of **BOTH** **calibration** and **accuracy**

### Negative Log-Likelihood (NLL)

$$\text{NLL} = -\frac{1}{N} \sum_{n=1}^N y_n\log( p(y_n)  + (1-y_n)\log(1- p(y_n) )$$

- $\hat{p}$ is predicted probability
- $y$ is the true class
- Lower NLL indicates better calibration and accuracy of the uncertainty estimates.

> NLL is a measure of **BOTH** **calibration** and **accuracy**

## Misclassification

In the binary classification setting, an uncertainty threshold is employed to differentiate between positive (i.e., certain) and negative (i.e., uncertain) samples. The outcomes of this classification are then compared against the true labels of each sample, which can be categorized as either correct or incorrect. From this perspective, a confusion matrix can be constructed, distinguishing four possible cases with the following counts:

- True Positive (TP): The prediction is uncertain, and the expected label and the prediction differ
- False Negative (FP): The prediction is certain, but the expected label and the prediction differ
- True Negative (TN): The prediction is certain, and the expected label and the prediction are identical
- False Negative (FN): The prediction is uncertain, but the prediction and the expected label are identical

![](images_uq/misclass.png)

For 

- $L_i = \mathcal{1} (\hat{y} = y)$
- $c(x)$ is uncertainty estimate

![](images_uq/misclass2.png) ![](images_uq/misclass3.png)

Since the optimal threshold is often unknown, we can evaluate performance using the area under the curve (AUC), which assesses all possible thresholds.

- The Area Under the Curve (AUC), whether for the Receiver Operating Characteristic (ROC) or Precision-Recall (PR) curves, serves as a comprehensive summary metric for model performance across varying thresholds.

### Out-of-Distribution (OOD) and Corruption Detection

This methodology can also be applied to out-of-distribution (OOD) detection, where we assume that OOD data is characterized by high uncertainty. In this context, positive samples are defined as OOD, while negative samples are classified as in-distribution (ID). By leveraging uncertainty estimates, we can effectively identify samples that deviate from the expected distribution, thereby enhancing the robustness of the classification model in real-world scenarios. This approach not only improves the model's reliability but also aids in mitigating the risks associated with misclassifying OOD data.

## Interval evaluations

### Coverage probability

$$P(y_l< y < y_u)$$

- Coverage Probability measures the proportion of prediction intervals that successfully contain the true label y.
- A higher coverage probability indicates a more reliable UQ method, as it suggests that the intervals are accurately capturing the true values.

### Interval Width

Interval Width w is calculated as: $w=y_u​−y_l​$

- A narrower interval indicates more precise predictions, while a wider interval suggests greater uncertainty.
- Balance between interval width and coverage probability is crucial.

# Predictive uncertainty model

As a modeller, one is mainly interested in the uncertainty that is propagated onto a prediction $\hat{y}$ , the so-called predictive uncertainty. Within the data acquisition model, the probability distribution for a prediction $\hat{y}$ based on some sample $x$ is given by

$$p(y|x, D) = \int p(y|x, \theta) p(\theta | D) d \theta $$

1. Epistemic uncertainty
    - “I am not sure because I have not seen it before.” 
    - Model uncertainty
2. Aleatoric uncertainty
    - “I have experienced it before, I know what I am doing, but I think there is more than one good answer to your question, so I cannot choose just one.”
    - Data uncertainty

![](images_uq/epis.png)

Yang, CI., Li, YP. Explainable uncertainty quantifications for deep learning-based molecular property prediction. J Cheminform 15, 13 (2023). https://doi.org/10.1186/s13321-023-00682-3

![](images_uq/vs.png)

Alex Kendall and Yarin Gal. What Uncertainties Do We Need in Bayesian Deep Learning for Computer Vision? 2017. doi: 10.48550/ARXIV.1703.04977. url: https://arxiv.org/abs/1703.04977.

## Epistemic uncertainty

- “I am not sure because I have not seen it before.” 
- Model uncertainty

![](images_uq/epis2.png) ![](images_uq/epis3.png)

$$p(y|x, D) = \int p(y|x, \theta) p(\theta | D) d \theta $$

$$p(\theta | D) \propto p(\theta )p(D|\theta)$$

- There are several possible classifiers compatible with the data we have observed (we are unsure about our prediction because several models could fit the training data).
- We wish to sample data from underexplored regions to increase our certainty in the choice of the model.

**Prior Hypothesis space $p(\theta)$**

![](images_uq/epis_prior.png)

Epistemic uncertainty cause by

1. The size of our training set is too small, and so the variance of our estimator is too high. 
2. The training data distribution does not cover some meaningful regions in the input space; there are some underexplored areas eg Out of distribution, distribution shift.

Note: Epistemic uncertainty is reducible. It stems from limitations in current knowledge and can be mitigated through data collection or model improvement.

#### Example: Distribution shift

![](images_uq/epis_ex.png)

## Aleatoric uncertainty

- “I have experienced it before, I know what I am doing, but I think there is more than one good answer to your question, so I cannot choose just one.”
- Data uncertainty

![](images_uq/alea.png)

Battleday et al. 2019 “Improving machine classification using human uncertainty measurements” https://openreview.net/pdf?id=rJl8BhRqF7

Aleatoric uncertainty, also known as data uncertainty, arises from information loss or noise in the data itself. Unlike epistemic uncertainty, which can be reduced by improving the model or adding more data, aleatoric uncertainty is irreducible—it represents uncertainty that is intrinsic to the data and cannot be eliminated through model adjustments.

![](images_uq/alea2.png)

Vaishaal Shankar, Rebecca Roelofs, Horia Mania, Alex Fang, Benjamin Recht, and Ludwig Schmidt. 2020. Evaluating machine accuracy on ImageNet. In Proceedings of the 37th International Conference on Machine Learning (ICML'20), Vol. 119. JMLR.org, Article 801, 8634–8644.

In image there are many object such as screen and coffee mug. Model may learn the association between object and cannot sure whather the correct answer is.

![](images_uq/alea3.png)

Vaishaal Shankar, Rebecca Roelofs, Horia Mania, Alex Fang, Benjamin Recht, and Ludwig Schmidt. 2020. Evaluating machine accuracy on ImageNet. In Proceedings of the 37th International Conference on Machine Learning (ICML'20), Vol. 119. JMLR.org, Article 801, 8634–8644.

You not sure whether it is Rabbit or Duck

![](images_uq/alea4.png)

Battleday et al. 2019 “Improving machine classification using human uncertainty measurements” https://openreview.net/pdf?id=rJl8BhRqF7

Missing features can introduce overlaps in two classes. When we have both features x1 and x2, points of the same label are well separated. For example, we know all the pixels in the N-digit MNIST example. We have no aleatoric uncertainty. As soon as we remove some features (e.g., a part of the image), we have overlaps between the classes.

![](images_uq/alea5.png)

An Y, Lam HK, Ling SH. Auto-Denoising for EEG Signals Using Generative Adversarial Network. Sensors. 2022; 22(5):1750. https://doi.org/10.3390/s22051750 

EEG sensor noise.



## Epistemic vs Aleatoric uncertainty

![](images_uq/alea_epis.png)

Hüllermeier E, Waegeman W. Aleatoric and epistemic uncertainty in machine learning: an introduction to concepts and methods. Mach Learn 2021;110:457–506.


![](images_uq/alea_epis2.png)

Gawlikowski J, Tassi CRN, Ali M, Lee J, Humt M, Feng J, et al. A survey of uncertainty in deep neural networks. Artif Intell Rev 2023;56:1513–89.

### Source of uncertainty in practice

1. the data: The occurrence of some information in the environment (e.g. a bird’s singing) and a measured observation of this information (e.g. an audio record). 
    - the variability in real world situations eg. out of distribution (Epistemic)
    - the errors inherent to the measurement systems (Aleatoric)
    - unknown data (Epistemic)
2. the DNN building process: The design and training of a neural network. (Epistemic)
    - Model architecture (Epistemic)
    - Model training process (Epistemic)

# Epistemic Uncertainty Quantification

Epistemic uncertainty represents uncertainty due to a lack of knowledge about the optimal model. It can be quantified using these idea:

1. Modeling Parameter Distributions: Estimating uncertainty in model parameters.
2. Out-of-Distribution (OOD) Detection: Assessing if data samples are dissimilar to the training set, often using distance metrics or anomaly detection techniques.

## Emsemble learning

In model ensembling, we train several deterministic models on the same (or subsampled) data simultaneously. We approximate $p(\theta | D)$ as ensemble of $M$ models.

$$p(\theta | D) \approx \frac{1}{M} \sum_{m=1}^M \delta(\theta - \theta^{m})$$

Then

$$\begin{split}
p(y|x, D) =& \int p(y|x, \theta) p(\theta | D) d \theta \\
\approx & \int p(y|x, \theta) \frac{1}{M} \sum_{m=1}^M \delta(\theta - \theta^{m}) d \theta \\
= & \sum_{m=1}^M\int p(y|x, \theta) \frac{1}{M} \delta(\theta - \theta^{m}) d \theta \\
= & \frac{1}{M}\sum_{m=1}^M p(y|x, \theta^{m})
\end{split}$$

This corresponds to averaging the predictions of individual models.

Pro:
- Conceptually simple – run the training algorithm M times and average outputs.
- Applicable to a wide range of models – from linear regression to ChatGPT.
- Parallelizable – if we have a lot of computational resources, we can train multiple models simultaneously on different cluster nodes (GPUs)
- Performant – ensembles are not only able to represent epistemic uncertainty but are also often more accurate.

Contra:
- Ensembles do not realize the full potential of Bayesian ML (no infinite number of models, no connectivity between the models).
- Space and time complexities scale linearly with M . If we have a limited number of GPUs, we must wait until the previous model finishes training
## Baysian methods

### MC dropout

Dropout, commonly used as a regularization technique, also serves as a form of Bayesian approximation when used during both training and testing phases. In this approach, we can interpret dropout as an approximation to a probabilistic model by drawing multiple "sampled" models through random network weight adjustments. This aligns with the Bayesian framework where dropout effectively **simulates an ensemble of models**.

At inference time, we can apply dropout in a way similar to model ensembling by performing multiple stochastic forward passes. The predictive distribution of the output 𝑦 given input 𝑥 and data 𝐷 is approximated as:

$$\begin{split}
p(y|x, D) =& \int p(y|x, \theta) p(\theta | D) d \theta \\
\approx& \frac{1}{K}\sum_{m=1}^K p(y|x, \theta^{k})
\end{split}$$

For $K$ is the number of times we run (samples) the models.


![](images_uq/mcdrop.png)

Balaji Lakshminarayanan, Alexander Pritzel, and Charles Blundell. Simple and Scalable Predictive Uncertainty Estimation using Deep Ensembles. 2016. url: https://arxiv.org/abs/1612.01474.

These days, MC dropout is treated as a method that does not really work.

### Variational inference

![](images_uq/bayes.png)

In variational inference, we assume that the posterior distribution of model parameters can be approximated by a simpler distribution, such as a Gaussian distribution

![](images_uq/var.png)

$$p(\theta | D) \sim \mathcal{N}(\theta| \mu,\sigma)$$

then

$$\begin{split}
p(y|x, D) =& \int p(y|x, \theta) p(\theta | D) d \theta \\
\approx & \int p(y|x, \theta)  \mathcal{N}(\theta| \mu,\sigma) d \theta \\
\approx& \frac{1}{K}\sum_{m=1}^K p(y|x, \theta^{k})
\end{split}$$

Similar to what we do in MCDropout, we can sample $\theta$ from normal distribution (approximate distribution).

# Aleatoric uncertainty quantification

To represent aleatoric uncertainty, we rely on two key components in our model design:

1. Model Architecture: Formulate a model architecture that accommodates multiple possible outputs. We should prepare, e.g., a probabilistic output where our model outputs the parameters of this output distribution rather than a single prediction.

2. Loss Function: Use a loss function that aligns the predicted output distribution with the observed data distribution. For example, in regression tasks, a negative log-likelihood loss can be used to train the model to fit the predicted distribution to the data, where higher predicted variance is encouraged for noisier data points. This approach ensures the model’s uncertainty estimates are consistent with the data variability.

## Loss function

Loss functions guide the model to make predictions that represent probability distributions over classes. A commonly used loss function in classification tasks is the Negative Log-Likelihood (NLL), or equivalently, cross-entropy loss:

$$L = -\sum_k y_k \log(f_k(x))$$

where $f_k(x)$ is the predicted probability for class 𝑘, and $y_k$ is the true label, often represented as a one-hot encoded vector.

- DNN is encouraged to predict $f_k(X)$ that correctly represents the spread $p(Y|X)$ in the training set.
- The popular softmax + cross-entropy design is already capable of handling aleatoric uncertainty!

### Calibration methods

However, these predicted probabilities $f_k(x)$ are not always well-calibrated, meaning they may not accurately reflect the true likelihood of an outcome. For instance, if a model predicts a 90% probability for a class, it should be correct about 90% of the time for that class. In practice, models—especially deep networks—often overestimate or underestimate their confidence.

Calibration aims to adjust the predicted probabilities to make them more reliable. Techniques like temperature scaling, Platt scaling, and isotonic regression adjust the model outputs to produce well-calibrated probabilities. For example:

1. Temperature Scaling
2. Platt Scaling
3. Isotonic Regression

**Temperature scaling**

Temperature scaling is a simple but effective post-processing calibration technique. It involves dividing the logits (the raw output of the neural network before softmax) by a temperature parameter 𝑇 before applying the softmax function:

$$\text{softmax}(\frac{z}{T})$$

![](images_uq/temp.png)

Guo C, Pleiss G, Sun Y, Weinberger KQ. On Calibration of Modern Neural Networks 2017.

**Platt Scaling**

![](images_uq/platt.png)

https://en.wikipedia.org/wiki/Platt_scaling

In binary classification, Platt scaling fits a **logistic regression model** on the raw outputs (logits) to map them to calibrated probabilities. For multi-class problems, it can be extended by applying it to each class independently in a one-vs-rest manner.

**Isotonic Regression**

![](images_uq/iso.png)

https://en.wikipedia.org/wiki/Isotonic_regression

Isotonic regression finds a piecewise constant function that best fits the validation data. Unlike temperature and Platt scaling, it does not assume a specific functional form (non-parametric). This makes isotonic regression very flexible, as it allows for highly nonlinear mappings.

## Mixture of Gaussians

Example:

In the training set, we will see many cases of mixed supervision. There will be many cases where a vehicle will take turns (1), (2), or (3), coming from the same direction. Therefore, we need something better than a single distribution to approximate the true multimodal conditional distribution. We have to accommodate all the complexity that can happen in the future.

![](images_uq/mog3.png)

A Mixture of Gaussians (MoG) is a probabilistic model that assumes data points are generated from a mixture of multiple Gaussian distributions, each with its own mean and variance. This model is particularly useful in cases where the data has complex, multimodal distributions that a single Gaussian cannot capture effectively.

![](images_uq/mog.png)

![](images_uq/mog2.png)

## Test‑time augmentation

Test-Time Augmentation (TTA) is a practical technique used in uncertainty quantification to improve predictions and measure model confidence by leveraging data transformations at **inference time**. Instead of predicting on a single test sample, TTA generates multiple augmented versions of the test data through transformations commonly applied during training, like cropping, flipping, rotating, or adjusting brightness. This allows the model to make predictions across slightly varied perspectives of the same input, yielding a distribution of predictions rather than a single output.


# Application in Medicine

![](images_uq/app.png)

Zhang X, Sisniega A, Zbijewski WB, Lee J, Jones CK, Wu P, et al. Combining physics‐based models with deep learning image synthesis and uncertainty in intraoperative cone‐beam CT of the brain. Medical Physics 2023;50:2607–24.


![](images_uq/ex.png)

Pereira T, Cardoso S, Guerreiro M, Mendonça A, Madeira SC. Targeting the uncertainty of predictions at patient-level using an ensemble of classifiers coupled with calibration methods, Venn-ABERS, and Conformal Predictors: A case study in AD. Journal of Biomedical Informatics 2020;101:103350.

